# Retrieval Augmented Generation

RAG combines search (retrieval) with AI text generation. First retrieve relevant documents, then use them to generate informed responses.

![images/llm_2_rag_basic.png](images/llm_2_rag_basic.png)

## Connect to Weaviate

Connect to a Weaviate instance with third party credentials as needed.

In [ ]:
# Refresh credentials & load the Weaviate IP
from helpers import update_creds

AWS_ACCESS_KEY, AWS_SECRET_KEY, AWS_SESSION_TOKEN = update_creds()

%store -r WEAVIATE_IP

In [ ]:
import weaviate

client = weaviate.connect_to_local(
    WEAVIATE_IP,
    headers = {
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SESSION_TOKEN,
    }        
)

client.is_ready()

### Start with (R) - Retrieval

First step: retrieve relevant documents using search.

In [ ]:
articles = client.collections.use("FinancialArticles")

response = articles.query.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
)

for item in response.objects:
    print(item.properties["article_title"])

### Add (AG) - augmented generation - to make full RAG

Second step: use retrieved documents to generate informed responses.

#### Single Prompt

> Generate a response per **retrieved** object.

In [ ]:
# Let's add some colour to our lives :)
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

In [ ]:
from weaviate.classes.generate import GenerativeConfig

articles = client.collections.use("FinancialArticles")

response = articles.generate.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
    generative_provider=GenerativeConfig.openai(
        model="gpt-4o-mini",
    ),
    single_prompt="Summarize this article to a sentence or two: {article}"
)

for item in response.objects:
    print(f"{BLUE}=== Source ===")
    print(item.properties["article_title"])

    print(f"{PURPLE}=== Generated Response ===")
    print(item.generative.text)
    print("\n")

#### Grouped Task

Generate one response using all retrieved documents together.

In [ ]:
response = articles.generate.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
    generative_provider=GenerativeConfig.openai(model="gpt-4o-mini"),
    grouped_task="Explain top 3 or so common trends or overall learnings in these articles. Please only use the provided content."
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

![images/llm_3_rag_weaviate.png](images/llm_3_rag_weaviate.png)

#### Specify which properties to use for grouped task

Control which document fields the AI uses for generation.

In [ ]:
response = articles.generate.near_text(
    query="artificial intelligence",
    limit=50,
    target_vector="title",
    grouped_task="What are the top 3 most common types of articles, based on these titles? Please only use the provided content.",
    grouped_properties=["article_title"],

    generative_provider=GenerativeConfig.openai(
        model="gpt-4o-mini",
    ),
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

## Set default Generative model

Configure a default model to avoid specifying it each time.

In [ ]:
from weaviate.classes.config import Reconfigure

articles.config.update(
    generative_config=Reconfigure.Generative.openai(
        model="gpt-4o-mini"  # Update the generative model
    )
)

Try generative query without specifying the model (uses default).

In [ ]:
response = articles.generate.near_text(
    query="cryptocurrency regulation",
    limit=5,
    target_vector="title",
    grouped_task="Briefly, what topics are covered in these articles? Please only use the provided content.",
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

## Close the client

Always close your connection when finished.

In [ ]:
client.close()